<a href="https://colab.research.google.com/github/babatundeibukun/fastai_deep_learning/blob/main/building_Learner_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#i need that data that contaons just 3's and 7's
#that is my first step
#importing all i need to make my life easy


#hide
from fastai.vision.all import *
#from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)


In [ ]:
path.ls()

(#3) [Path('/root/.fastai/data/mnist_sample/valid'),Path('/root/.fastai/data/mnist_sample/labels.csv'),Path('/root/.fastai/data/mnist_sample/train')]

In [ ]:
(path/'train').ls()


(#2) [Path('/root/.fastai/data/mnist_sample/train/7'),Path('/root/.fastai/data/mnist_sample/train/3')]

In [ ]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()


In [ ]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
len(three_tensors),len(seven_tensors)

(6131, 6265)

__STACKING TENSORS__


In [ ]:
stacked_threes = torch.stack(three_tensors).float()/255
stacked_sevens = torch.stack(seven_tensors).float()/255

In [ ]:
stacked_threes.shape, stacked_sevens.shape

(torch.Size([6131, 28, 28]), torch.Size([6265, 28, 28]))

__CONVERT THEM TO RANK 2 TENSORS__

In [ ]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_x.shape

torch.Size([12396, 784])

__CREATE TRAIN_Y__

In [ ]:
train_y = tensor([1]*len(stacked_threes) + [0]*len(stacked_sevens)).unsqueeze(1)
train_y.shape

torch.Size([12396, 1])

__CREATE OUR DATASET__

In [ ]:
ds = list(zip(train_x, train_y))
x, y = ds[0]
x.shape, y

(torch.Size([784]), tensor([1]))

In [ ]:
dl = DataLoader(ds , batch_size = 256)
xb, yb = first(dl)
xb.shape, yb.shape

(torch.Size([256, 784]), torch.Size([256, 1]))

__VALIDATION DATASET__

In [ ]:
valid_3_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255
valid_3_tens.shape,valid_7_tens.shape

(torch.Size([1010, 28, 28]), torch.Size([1028, 28, 28]))

In [ ]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

In [ ]:
valid_dl = DataLoader(valid_dset, batch_size=256)

__STEP 1 : INITIALIZE WEIGHTS FUNCTION__

In [ ]:
def init_params(size, std=1.0):
    return (torch.randn(size)*std).requires_grad_()

__STEP 2: PREDICTION FUNCTION__

In [ ]:
def linear1(xb):
    preds = xb@weights + bias
    return preds

__STEP 3 : LOSS FUNCTION__

In [ ]:
def mnist_loss(preds, targets):
    preds = preds.sigmoid()
    return torch.where(targets==1.0, 1-preds, preds).mean()

__STEP 4 : CALC GRAD__

In [ ]:
def calc_grad(xb, yb , model):
    preds = model(xb)
    loss= mnist_loss(preds, yb)
    loss.backward()

__STEP 5 : BATCH ACCURACY__

In [ ]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    corrects = (preds > 0.5).float() == yb
    return corrects.float().mean()


__STEP 5 : VALIDATION_ACCURACY FUNCTION__

In [ ]:
def validation_accuracy(model):
    accs = [batch_accuracy(model(xb), yb) for  xb, yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

__STEP 5 : TRAIN EPOCH FUNCTION__

In [ ]:
def train_epoch(model, params , lr):
    for xb, yb in dl:
        calc_grad(xb, yb, model)
        for p in params:
            p.data -= p.grad *lr
            p.grad.zero_()

In [ ]:
weights = init_params(28*28, 1)
bias = init_params(1)
weights.shape, bias.shape

(torch.Size([784]), torch.Size([1]))

In [ ]:
lr=1.0
params = weights, bias
train_epoch(linear1, params, lr)
validation_accuracy(linear1)

0.5691

In [ ]:
for i in range(20):
    train_epoch(linear1, params, lr)
    print(validation_accuracy(linear1), end = ' ')

0.8104 0.9052 0.9314 0.9431 0.9469 0.9489 0.9498 0.9512 0.9532 0.9551 0.9571 0.9585 0.959 0.96 0.9595 0.96 0.961 0.9615 0.962 0.9624 

__STEP 6 : BASIC OPTIMIZATION STEP__

In [ ]:
linear_model = nn.Linear(28*28,1)
weights, bias =linear_model.parameters()
weights.shape, bias.shape

(torch.Size([1, 784]), torch.Size([1]))

In [ ]:
class BasicOptim:
    def __init__(self, params , lr):
        self.params = list(params)
        self.lr = lr

    def step(self):
        for p in self.params:
            p.data -= p.grad *self.lr

    def zero_grad(self):
        for p in self.params:
            p.grad.zero_()

In [ ]:
linear_model = nn.Linear(28*28,1)
weights, bias =linear_model.parameters()
weights.shape, bias.shape

(torch.Size([1, 784]), torch.Size([1]))

In [ ]:
opt = BasicOptim(linear_model.parameters(), lr)


In [ ]:
def train_epoch(model):
    for xb, yb in dl:
        calc_grad(xb, yb, model)
        opt.step()
        opt.zero_grad()

In [ ]:
train_epoch(linear_model)
validation_accuracy(linear_model)

0.4932

In [ ]:
def train_model(model, epoch):
    for i in range(epoch):
        train_epoch(model)
        print(validation_accuracy(model), end = ' ')



In [ ]:
train_model(linear_model, 20)

0.7876 0.8491 0.9135 0.934 0.9482 0.955 0.9633 0.9658 0.9672 0.9702 0.9716 0.9736 0.975 0.976 0.9765 0.9775 0.9775 0.9785 0.9789 0.9789 

In [ ]:
linear_model = nn.Linear(28*28,1)
weights, bias =linear_model.parameters()
weights.shape, bias.shape

(torch.Size([1, 784]), torch.Size([1]))

In [ ]:
#fast ai gives a substitute for BasicOptim which is .....SGD
opt = SGD(linear_model.parameters(), lr)

In [ ]:
train_model(linear_model, 20)

0.4932 0.8217 0.8466 0.9126 0.9336 0.9472 0.9555 0.9628 0.9662 0.9677 0.9697 0.9716 0.9726 0.9745 0.9755 0.9765 0.9775 0.978 0.9785 0.9785 

__BUILD A LEARNER__

In [ ]:
dls = DataLoaders(dl, valid_dl)

In [ ]:
learn = Learner(dls, nn.Linear(28*28, 1), loss_func= mnist_loss, opt_func=SGD, metrics= batch_accuracy)

In [ ]:
learn.fit(20, lr=1)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.637105,0.503440,0.495584,00:00
1,0.503201,0.203106,0.824338,00:00
2,0.185543,0.174588,0.842983,00:00
3,0.081607,0.105023,0.910697,00:00
4,0.043349,0.077123,0.932777,00:00
5,0.028429,0.061905,0.947498,00:00
6,0.022291,0.052386,0.955839,00:00
7,0.019554,0.046055,0.962709,00:00
8,0.018162,0.041613,0.965162,00:00
9,0.017325,0.038343,0.967615,00:00


__BUILDING A SIMPLE NET__

In [ ]:
simple_net = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 1)
)

In [ ]:
learn = Learner(dls, simple_net, loss_func=mnist_loss, opt_func=SGD, metrics = batch_accuracy)


In [ ]:
learn.fit(40, 0.1)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.319893,0.416680,0.504416,00:00
1,0.150109,0.232627,0.797841,00:00
2,0.082693,0.115986,0.917076,00:00
3,0.054039,0.078070,0.940628,00:00
4,0.040809,0.060888,0.955348,00:00
5,0.034079,0.051257,0.963199,00:00
6,0.030226,0.045180,0.964181,00:00
7,0.027721,0.041027,0.966143,00:00
8,0.025914,0.037991,0.969087,00:00
9,0.024510,0.035671,0.970559,00:00


In [ ]:
dls = ImageDataLoaders.from_folder(path)
learn = vision_learner(dls, resnet18, pretrained=False,
                    loss_func=F.cross_entropy, metrics=accuracy)
learn.fit_one_cycle(1, 0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.090304,0.007439,0.997056,01:48
